<a href="https://colab.research.google.com/github/jhmartel/fp/blob/master/_notebooks/2022-03-16-test_upload.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Test
"Test. "

toc: false
branch: master
badges: false
comments: true
author: JHM
categories: [einstein, maxwell, wave equation, lorentz, SR]

This article is an extension of Don Scott's work [ref] on modelling Birkeland currents. This is low-beta plasma delivering electric current along filaments in space. [ref]. Scott's model relies on the Maxwell equations and Lorentz force law. But these equations are undergoing a renewed criticism, and Weber's electrodynamics is being reconsidered as an alternative. Our objective in this article is to derive the equations of Birkeland currents based on Weber's electrodynamics.

# Ampere's Force Law

Ampere's investigations in 1820-1826 led to his proposing a force law between two current elements $I_1 ds_1$ and $I_2 ds_2$. The current element has a scalar $I$ which represents the net current intensity, and $ds$ is a differential vector element representing arc length along the circuit. The magnitude of $I$ typically represents the *current intensity*, while the magnitude of $ds$ represents the *current velocity*. 

Ampere determined that the force between two current elements was radial and proportional to the current intensity and to current velocities. Thus the force $F$ was proportional to $I_1 I_2 ds_1 ds_2$. Ampere discovered the inverse square distance proportionality. And moreover determined the precise coefficient of proportionality. Thus in cgs units Ampere's force law is expressed:
 
$$ F_{~I_2 ds_2 ~~ \text{on} ~~ I_1 ds_1 } =F_{21}= - I_1 I_2\frac{\hat{r}_{12}}{r_{12}^2} [ 2(ds_1 \cdot ds_2) - 3 (\hat{r}_{12}\cdot ds_1) ( \hat{r}_{12}\cdot ds_2) ] =-F_{12} .$$


The following code uses Ampere's formula:

In [ ]:
# Vector dot product and scalar product required.
def dot(v1, v2):
    vx1, vy1, vz1 = v1
    vx2, vy2, vz2 = v2
    return vx1*vx2+vy1*vy2+vz1*vz2

def scalar(a, v1):
    vx1, vy1, vz1 = v1 
    return [a*vx1, a*vy1, a*vz1]

## Ampere's Force 
def F(state1, state2):
    x1, y1, z1, vx1, vy1, vz1 = state1
    x2, y2, z2, vx2, vy2, vz2 = state2
    v1=[vx1,vy1,vz1]
    v2=[vx2,vy2,vz2]

    I1=dot(v1,v1)**0.5
    I2=dot(v2,v2)**0.5

    rho_12 = ((x2-x1)**2+(y2-y1)**2+(z2-z1)**2)**0.5
    rhat12 = scalar(rho_12**-1, [x1-x2, y1-y2, z1-z2])

    coefficient = -I1*I2*(2*dot(v1,v2)-3*dot(rhat12, v1)*dot(rhat12, v2))*(rho_12)**-1
    return [x1,y1,z1]+scalar(coefficient, rhat12)


s0=[0,0,0,.1,.7,0]
s1=[1,0,0,0.9,0,0]    

F(s0,s1)


[0, 0, 0, -0.05727564927611038, 0.0, 0.0]

Ampere's force law has several predictions. 

*   Parallel colinear current elements repel
*   Reversing a parallel colinear current attracts by 2/3rd of the repelling force.
*   Parallel adjacent current elements attract

In [ ]:
# Parallel Colinear Current Elements Repel
s0=[ 0,0,0, 0,0,1 ]
s1=[ 0,0,2, 0,0,1 ]
s3= [ 0,0,3, 0,0,-1 ]
print(F(s0,s1))
print(F(s1,s0))


print(F(s0,s3))
print(F(s3,s0))

# Parallel Adjacent Current Elements Attract
s2=[ 1,0,0, 0,0,1]
print(F(s0, s2))

[0, 0, 0, 0.0, 0.0, -0.5]
[0, 0, 2, 0.0, 0.0, 0.5]
[0, 0, 0, -0.0, -0.0, 0.3333333333333333]
[0, 0, 3, -0.0, -0.0, -0.3333333333333333]
[0, 0, 0, 2.0, -0.0, -0.0]


We want to move towards the Birkeland currents, which is a cylindrical plasma. Following Scott's approach, we look for an integral equation in the cylindrical coordinates $\rho$, $\theta$, $z$. 

In [ ]:
import numpy as np

# Converts Cartesian states to Cylindrical States.
def C(state):
    x,y,z,vx,vy,vz = state
    
    rho=(x*x+y*y)**0.5
    theta=np.arctan(y/x)
    
    vr = (vx*x+vy*y)/rho
    vt = (-vx*y+vy*x)*(rho**-2)

    
    return [ rho,theta,z, vr,vt,vz ]

# Converts Cylindrical State to Cartesian state
def D(state):
    rho,theta,z,vr,vt,vz = state

    x=rho*(np.cos(theta))
    y=rho*(np.sin(theta))

    vx=vr*(np.cos(theta))-vt*(np.sin(theta))*rho
    vy=vr*(np.sin(theta))+vt*(np.cos(theta))*rho

    return [ x,y,z, vx,vy,vz ]
    

#print(C([ 1,1,0, 0,0.5,1.2]))
#print(D(C([ 1,1,0, 0,0.5,1.2])))

## Ampere's Force in Cylindrical Coordinates rho, theta, z:

def AFC(state1, state2):
    rho1, theta1, z1, vr1, vt1, vz1 = state1 
    return C(F(D(state2), D(state1)))

# output of above force is cartesian valued. 
# We need to translate the vector back to the state which is centred on state1 position.
c0=[ 1.1,0,0, 0,1.2,1 ]  ## [ rho, theta, z, vr, vt, vz ]
c1=[ 1,0,0, 0,0,1 ]

print(AFC(c0, c1)[0:6])

## Need use sympy to obtain some formulas(!) for the force law, and obtain the integral equation for the equilibrium condition.



[1.0, 0.0, 0, 33.120386471175095, -0.0, -0.0]


In the previous cell we have a function AFC which computes Amperes force in cylindrical coordinates. But this does not help us symbolically identify the balance equation [ref] in cylindrical coordinates.  

[1.1, 0, 0, 33.120386471175095, -0.0, -0.0]

If $C_2$ is a closed current with line element $ds_2$, then the force of the current on the line element is $$F_{C_2 ~~\text{on}~~I_1 ds_1} =  I_1 ds_1 \times ( \oint_{C_2} I_2 ds_2 \times \nabla_1(\frac{1}{r_{12}})).$$

N.B. Ampere's force between current elements is radial. However the force exerted by a closed current $C$ on a current element $I_1 ds_1$  is always orthogonal to the current element, as is clear from the above equation. This was announced by Ampere in 1820s [ref]. Historically one finds that Ampere's force law differs from Grassmann's proposed force law by an exact differential, hence the integrals over closed curves coincide. [ref: Assis]. Grassmann's force law was the more formulaic  $$F^G_{21}:=I_1 d\ell_1 \times (I_2 d\ell_2 \times \nabla_1(\frac{1}{r_{12}})) . $$ However this law does *not* satisfy action-reaction principle, except between closed currents. In general for current elements we have $F_{21} \neq F_{12} = 0$. 

When the currents are filamentary, then Ampere's law yields a virtual magnetic field $B$ defined by $$B=\oint_{C_2} I_2 d\ell_2 \times \nabla_1(\frac{1}{r}) .$$ However we can replace $I_2 d\ell_2$ with $J dVol_2$, where $Vol_2$ represents a volume form defined relative to the index $2$ variable. Then we have the following expression from [Assis, Ch.4]: $$B(r_1,t)=\iiint d^3(r_2)~ J(r_2,t) \times \nabla_1 (\frac{1}{r_{12}}).$$

 
 
 
 
 
In the context of Birkeland currents, we have a three-dimensional cylinder which is assumed to contain plasma [no collisional low beta] and carrying an electric current between it's two ends. The magnetic field $B$ in this setting induced by the charge distribution $J=J(x)$ is computed by Ampere's force as a volume integral over the tube.

$$B(x)=\int_{Tube} d^3(y) ~~ \bf{F}_{J(y) ~ \text{on}~J(x)}$$

Remark. In basic electromagnetism theory after Maxwell, it's important to observe that the Maxwell magnetic field $B(x)$ can be nonzero at points where $J(x)=0$. I.e., it is supposed that $B$ can be nonzero at points in vacuum where there is no charge present. But of course, $B$ properly defined depends on a choice of *test* charge, i.e. $B$ represents an amount of force that would be exerted on the test charge *if* that charge was located at $x$. But of course then $B(x)$ represents not a vacuum at $x$ but implicitly involves a test current element to exist at $x$. This is perhaps overly philosophical remark.

Is it possible that $B(x)$ is nonzero at a point where $J(x)=0$ ?  No, because we cannot compute $B(x)$ at a point where $J(x)=0$ except to compute $B=0$. This is because we are expressly considering the interaction of $J(x)$ with $J(y)$ at a different point. We assume there is a radial action-at-a-distance between the charges. Therefore the equilibrium condition for the field $J$ in the Birkeland current is $$\int dy ~~ \bf{F}_{J(y)~\text{on}~ J(x)}=0$$ for every $x$. The above integral represents a Bochner vector-valued integral. It therefore represents three equations in the $x,y,z$ rectilinear coordinates. 

Following Don Scott, we can express these three equations using cyclindrical coordinates, and making some general assumptions about the variation of $B$ and $J$ with respect to time $t$ and the cylindrical coordinates $\rho, \theta, z$. 

However, how do we guarantee that current is being delivered, i.e. that the Birkeland current *is* delivering power? 


To compute the internal pressures of the charge distribution $J$ requires a method for determining the interaction of $J$ and $B$. Now Lorentz force law would imply that $B$ acts on $J$ by the cross-product $B\times J$. However we prefer to follow the Ampere-Weber electrodynamics. In this case we have *no* law for the interaction of $B$ and $J$. What is required is a different integration, namely $$\iiint dy~~~ \bf{F}

 computing $\bf{F}_{B(x)~ \text{on}~ J(x)}.$


The magnetic fields $B$ then became the integral of forces between currents. As Assis has proved in [ref] Ampere's formula establishes a computational basis for the non-existence of magnetic monopoles. In otherwords we find that $$\nabla \cdot B = \nabla \cdot (\nabla \times [potential])=0$$ since the divergence of a curl vanishes identically, i.e. $d\circ d=0$ where $d$ is the exterior differential.

In otherwords, Ampere proved that the magnetic field $B$, Faraday's lines of force, were always generated by currents. But which currents? If we include image of classic magnet with the iron filings, then where is the current that is orientating the magnet, and what "current" corresponds to the iron filings? 



How do we model a Birkeland current? [Include image]
We imagine a cylinder $D$ filled with plasma, and delivering a steady electrical current between the two ends of the cylinder. The question is: what is the charge distribution of a current-carrying plasma cylinder which has the minimal amount of internal *pressure*. 

If $J$ represents the current density, then the equations of a Birkeland current with balanced internal forces satisfies $$0= \int_D d^3(y) ~ F_{J(y)~~\text{on}~~ J(x)}$$ for all $x\in D$. 

Following Don Scott's approach, we look to express the above balanced equation in terms of cylindrical coordinates.

#hide 


 Weber proposed that every current element $I d \ell $ was equal to some amount of charge in motion, something like $I d\ell = q v$ or $I d\ell = \sum q_i v_i$. Weber assumed [ref] that the current elements consisted of a combination of charge-velocities whose total electrical charge was zero $\sum q_i=0$ for each current element. The net zero electric charge for each current element still leaves the velocities of the charges independant from one another. If one assumes that $I d\ell$ is a sum of $v_+$ and $v_-$ particles with charges $q_+ + q_-=0$. 
